In [1]:
import os
import asyncio
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""
os.environ["OPENAI_API_KEY"] =""
os.environ["SERPAPI_API_KEY"] = ""

In [1]:
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain import ConversationChain
from langchain.agents import initialize_agent, load_tools, AgentType

In [ ]:
from langchain.chains.

In [2]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.prompts import FewShotPromptTemplate

from langchain.document_loaders import HNLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

from langchain.chains import SimpleSequentialChain
from langchain.chains.summarize import load_summarize_chain

In [5]:
llm = OpenAI(model = 'text-davinci-003',temperature = 0.5)
text = "what do you think about God?"
print(llm.invoke(text))

AuthenticationError: Incorrect API key provided: sk-VaIB9***************************************fLQQ. You can find your API key at https://platform.openai.com/account/api-keys.

In [14]:
def generate_serially(text):
    for _ in range(10):
        output = llm.invoke(text)
        print(output)

async def generate_async(text):
    output = await llm.ainvoke(text)
    print(output)

async def generate_concurrently(text):
    task = [generate_async(text) for _ in range(10)]
    await asyncio.gather(*task)

In [16]:
#generate_serially(text)
#await generate_concurrently(text)

In [20]:
prompt = PromptTemplate(input_variables= ["name", "age"],
                        template = "What are the characterics of a someone around the age of {age} living {name}")
print(llm.invoke(prompt.format(name = 'califonia', age = 34)))



1. Likely to be well-educated, with a college degree or higher. 
2. Likely to be employed in a professional or managerial role. 
3. Likely to be married and/or have children. 
4. Likely to be technologically savvy and have access to the latest technology. 
5. Likely to be socially active and engaged in their community. 
6. Likely to be financially secure and have access to a range of investment and savings options. 
7. Likely to be health-conscious and take part in regular exercise and healthy eating. 
8. Likely to be environmentally aware and take part in activities that reduce their carbon footprint.


In [18]:
prompt = PromptTemplate(input_variables = ["adjective"],
                        template = "What are some common characteristics of {adjective}")
print(llm.invoke(prompt.format(adjective = "God")))

?

1. Omnipotent: God is all-powerful and can do anything.

2. Omniscient: God knows everything.

3. Omnipresent: God is everywhere at once.

4. Just: God is fair and just, and He will always do what is right.

5. Loving: God is loving and cares for all of His creation.

6. Holy: God is completely perfect and pure.

7. Merciful: God is compassionate and forgiving.

8. Self-existent: God has no beginning or end.

9. Sovereign: God is the ruler of all and is in control of all things.


In [25]:
prompt = PromptTemplate(input_variables= ["description"],
                        template = "We are a telecommunication company and we are trying to prevent customers from churning\
                            . Generate a promotional mail for one of our customer using information in {description} as \
                                context and it should have a subject relating to the context")
context = "Generate a mail for one of our customers in a relationship informing them of a network upgrade coming to the area soon"
print(llm.invoke(prompt.format(description = context)))



Subject: Get Ready for Faster Network Speeds

Dear [Name], 

We hope you're doing well and enjoying your relationship with us at [Company Name]. We're excited to let you know that we have great news about a network upgrade coming to your area soon. 

This upgrade will mean faster download and upload speeds for you, so you can enjoy a better experience when streaming, downloading, and browsing the web. You'll also have improved access to our services and features. 

We know you'll be thrilled with the improved network speeds, so we wanted to let you know right away. We hope you'll take advantage of the new network upgrade and enjoy the improved performance it offers. 

Thanks for being a loyal customer. 

Sincerely, 
[Company Name]


In [30]:
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run("fish"))

?

1. Scales 
2. Gills 
3. Backbone 
4. Fins 
5. Tail 
6. Adaptable to different environments 
7. Cold-blooded 
8. Lay eggs or bear live young


In [40]:
tools = load_tools(['serpapi', 'llm-math'], llm=llm)
agent = initialize_agent(tools, llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)
agent.invoke("who is the current leader of Japan? What is the larget prime number that is smaller than the president age")



> Entering new AgentExecutor chain...
 I need to find out who the current leader of Japan is and what the largest prime number is that is smaller than the president age
Action: Search
Action Input: "Current leader of Japan"
Observation: Fumio Kishida is the current prime minister of Japan, replacing Yoshihide Suga on 4 October 2021. As of 28 October 2023, there have been 64 individual prime ministers serving 101 terms of office.
Thought: I need to find the largest prime number smaller than the president's age
Action: Calculator
Action Input: 64
Observation: Answer: 64
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-YU2PIW6xmwCHzFyh6xWS1gII on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-YU2PIW6xmwCHzFyh6xWS1gII on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals

 I now know the final answer
Final Answer: Fumio Kishida is the current prime minister of Japan and the largest prime number smaller than his age is 64.

> Finished chain.


{'input': 'who is the current leader of Japan? What is the larget prime number that is smaller than the president age',
 'output': 'Fumio Kishida is the current prime minister of Japan and the largest prime number smaller than his age is 64.'}

In [41]:
conversation = ConversationChain(llm = llm, verbose = True)
conversation.invoke(input = "Hi there")

In [43]:
conversation.invoke(input = "i'm doinf fine. How are you doing yourself")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there
AI:  Hi! How are you doing today?
Human: i'm doinf fine. How are you doing yourself
AI:

> Finished chain.


{'input': "i'm doinf fine. How are you doing yourself",
 'history': 'Human: Hi there\nAI:  Hi! How are you doing today?',
 'response': " I'm doing great! I'm excited to chat with you. What would you like to talk about?"}

In [51]:
prompt = PromptTemplate(
    input_variables = ["input", "output"],
    template = "A {input} if to a {output}"
)

examples = [
    {"input": "pirate", "output": "ship"},
    {"input":"pilot", "output":"plane"}
]

In [58]:
selector = SemanticSimilarityExampleSelector.from_examples(examples,
                                                           OpenAIEmbeddings(),
                                                           FAISS,
                                                           k=2)

In [60]:
similar_prompt = FewShotPromptTemplate(
    example_selector=selector,
    example_prompt=prompt,
    prefix="Give the location an item can be found",
    suffix = "input: {noun}\n output:",
    input_variables=['noun']
)

noun = "student"
print(llm.invoke(similar_prompt.format(noun = noun)))

In [64]:
print(similar_prompt.format(noun = noun))

Give the location an item can be found

A pilot if to a plane

A pirate if to a ship

input: student
 output:


In [67]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 150,
    chunk_overlap = 20
)

In [ ]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(text, embedding = embeddings)
retrieval = db.as_retriever()
embedding_list = embeddings.embed_documents()

In [33]:
chat_history = ChatMessageHistory()
llm = ChatOpenAI(temperature = 0.5)

user_input = input("Enter your message:",)
for i in range(5):

    chat_history.add_user_message(user_input)
    response = llm(chat_history.messages)
    print(user_input)
    print(response.content)
    chat_history.add_ai_message(response.content)

hey
Hello! How can I assist you today?
hey
Hello! How can I assist you today?
hey
Hello! How can I assist you today?


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-idkrbVSuQwUhcUDmzLJNv4nO on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


KeyboardInterrupt: 

In [ ]:
history = ChatMessageHistory()
history.add_ai_message()
history.add_user_message()